<a href="https://colab.research.google.com/github/rohitshekar/cancer/blob/main/lunganadcolon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle/Kaggle"

In [3]:
%mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/Kaggle/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
%cd /content/

/content


In [7]:
%mkdir images
%cd images

/content/images


In [8]:
!kaggle datasets download -d obulisainaren/multi-cancer

100% 8.60G/8.62G [01:38<00:00, 130MB/s]
100% 8.62G/8.62G [01:38<00:00, 93.8MB/s]


In [9]:
!unzip multi-cancer.zip

Streaming output truncated to the last 5000 lines.
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0002.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0003.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0004.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0005.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0006.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0007.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0008.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0009.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0010.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0011.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0012.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0013.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0014.jpg  
  inflating: Multi Cancer/Oral Cancer/oral_scc/oral_scc_0015.jpg  
  inflating

In [11]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,validation_split=0.2)
test_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)


In [14]:
train_set = train_datagen.flow_from_directory('/content/images/Multi Cancer/Lung and Colon Cancer',
                                                 target_size = (128, 128),
                                                 batch_size = 32,seed=42,
                                                 class_mode = 'categorical',subset='training')
test_set = test_datagen.flow_from_directory('/content/images/Multi Cancer/Lung and Colon Cancer',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='validation',seed=42)

Found 20000 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.


In [15]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [18]:
inputs = Input((128,128,3))
# convolutional layers
conv_1 = Conv2D(32, kernel_size=(2, 2), activation='relu',kernel_regularizer=regularizers.l2(l=0.01),kernel_initializer='he_normal') (inputs)
maxp_1 = MaxPooling2D(pool_size=(2, 2)) (conv_1)
conv_2 = Conv2D(64, kernel_size=(2, 2), activation='relu',kernel_regularizer=regularizers.l2(l=0.01),kernel_initializer='he_normal') (maxp_1)
maxp_2 = MaxPooling2D(pool_size=(2, 2)) (conv_2)
conv_3 = Conv2D(128, kernel_size=(2, 2), activation='relu',kernel_regularizer=regularizers.l2(l=0.01),kernel_initializer='he_normal') (maxp_2)
maxp_3 = MaxPooling2D(pool_size=(2, 2)) (conv_3)
conv_4 = Conv2D(256, kernel_size=(2, 2), activation='relu',kernel_regularizer=regularizers.l2(l=0.01),kernel_initializer='he_normal') (maxp_3)
maxp_4 = MaxPooling2D(pool_size=(2, 2)) (conv_4)
conv_5 = Conv2D(512, kernel_size=(2, 2), activation='relu',kernel_regularizer=regularizers.l2(l=0.01),kernel_initializer='he_normal') (maxp_4)
maxp_5 = MaxPooling2D(pool_size=(2, 2)) (conv_5)
conv_6 = Conv2D(512, kernel_size=(2, 2), activation='relu',kernel_regularizer=regularizers.l2(l=0.01),kernel_initializer='he_normal') (maxp_5)
maxp_6 = MaxPooling2D(pool_size=(2, 2)) (conv_6)


flatten = Flatten() (maxp_6)

# fully connected layers
dense_1 = Dense(256, activation='relu') (flatten)



dropout_1 = Dropout(0.3) (dense_1)



output_1 = Dense(5, activation='softmax') (dropout_1)



model = Model(inputs=[inputs], outputs=[output_1])

model.compile(loss= ['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(x = train_set, validation_data = test_set, epochs =15)

Epoch 1/15
625/625 [==============================] - 223s 352ms/step - loss: 4.4314 - accuracy: 0.7338 - val_loss: 1.3576 - val_accuracy: 0.8482
Epoch 2/15
625/625 [==============================] - 204s 326ms/step - loss: 1.1877 - accuracy: 0.8189 - val_loss: 0.9687 - val_accuracy: 0.8710
Epoch 3/15
625/625 [==============================] - 220s 352ms/step - loss: 0.9529 - accuracy: 0.8437 - val_loss: 0.8378 - val_accuracy: 0.8706
Epoch 4/15
625/625 [==============================] - 203s 325ms/step - loss: 0.8239 - accuracy: 0.8620 - val_loss: 0.7572 - val_accuracy: 0.8684
Epoch 5/15
625/625 [==============================] - 202s 323ms/step - loss: 0.7452 - accuracy: 0.8660 - val_loss: 0.7402 - val_accuracy: 0.8590
Epoch 6/15
625/625 [==============================] - 201s 322ms/step - loss: 0.6725 - accuracy: 0.8802 - val_loss: 0.6681 - val_accuracy: 0.8688
Epoch 7/15
625/625 [==============================] - 218s 349ms/step - loss: 0.6194 - accuracy: 0.8853 - val_loss: 0.5948 -

In [69]:
import numpy as np
from keras.preprocessing import image
results=[]
test_image = tf.keras.utils.load_img('/content/images/Multi Cancer/Lung and Colon Cancer/lung_scc/lung_scc_0010.jpg', target_size = (128,128))
test_image = tf.keras.utils.img_to_array(test_image)
test_image=test_image/255.0
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)
print(max(result[0]))


1/1 [==============================] - 0s 27ms/step
[[4.9555551e-06 2.6761897e-09 1.3821477e-02 3.5374859e-09 9.8617351e-01]]
0.9861735


In [20]:
model.save('/content/lungcoloncancer1.h5',save_format='h5')

In [ ]:
res=train_set.class_indices

{'cervix_dyk': 0,
 'cervix_koc': 1,
 'cervix_mep': 2,
 'cervix_pab': 3,
 'cervix_sfi': 4}